# 연습 문제


---
## 1. 데이터 API를 사용하는 이유는?
대용량 데이터셋을 읽고 효율적으로 전처리하기 위해.
- 다양한 소스에서 데이터를 적재하거나
- 동시에 여러 소스에서 데이터를 읽고
- 변환하고
- 레코드를 교대로 처리하고
- 데이터를 섞을 수 있음.
- 배치와 프리페치 기능도 있음


---
## 2. 대용량 데이터셋을 여러 파일로 나누면 어떤 장점이 있나?
- **셔플링 버퍼**를 사용해 데이터를 잘게 섞기 전에 크게 섞을 수 있음.  
- 또한 한 대의 컴퓨터에 담을 수 없는 아주 큰 데이터셋을 다룰 수 있음.
- 하나의 큰 파일보다 수천 개의 작은 파일을 다루는 것이 쉬움.
- 동시에 여러 서버에서 파일을 다운로드할 수 있어서 네트워크 밴드위스를 효율적으로 사용할 수 있음.


---
## 3. 학습 과정에서 입력 파이프라인의 병목을 어떻게 찾을 수 있는가? 어떻게 병목 현상을 고칠 수 있는가?
- **텐서보드**를 사용해 프로파일링 데이터를 시각화할 수 있음. 
- GPU가 완전히 활용되지 않고 입력 파이프라인에 병목현상이 보인다면 **여러 스레드에서 동시에 데이터를 읽고 전처리하여 몇 개의 배치를 프리페치하여** 해결할 수 있음.
- 이렇게 해도 GPU를 100% 사용하지 못한다면 전처리 코드를 최적화해볼 수 있음.
- 학습하는 동안 전처리를 동시에 할 필요가 없다면 데이터셋을 여러 개의 TFRecord로 저장하고 **미리 전처리**를 수행할 수 있음.


---
## 4. 어떤 이진 데이터도 TFRecord 파일 또는 직렬화된 프로토콜 버퍼로 저장할 수 있는가??
어떤 이진 데이터도 저장할 수 있음.



---
## 5. 모든 데이터를 Example 프로토콜 버퍼 포맷으로 변환해야 하는가? 자신만의 프로토콜 버퍼 정의를 사용하는 것은 어떤가?
Example 프로토콜 포맷만으로도 충분하여 **딱히 자신만의 포맷을 정의할 필요 없음**  


---
## 6. TFRecord 를 사용할 때 언제 압축을 사용해야 되는가? 왜 기본적으로 압축을 사용하지 않는가?
학습 코드에서 TFRecord 파일을 다운로드해야 할 때 **파일 크기를 줄이고 다운로드 시간을 줄이기 위해** 일반적으로 압축을 활성화함.  
하지만 **파일이 학습 스크립트와 같은 머신에 있으면** 압축을 해제하는 데 CPU자원을 쓰지 않도록 압축하지 않는 것이 좋음.


---
## 7.데이터 파일을 작성할 때, 또는 tf.data 파이프라인 안에서, 모델의 전처리 층에서, TF변환을 사용하여 데이터를 전처리할 수 있음. 각 방식의 장단점은?
> **데이터 파일을 작성할 때 전처리할 경우**  
>- 실시간으로 전처리할 필요가 없어져서 학습 스크립트가 빨리 실행됨.  
>- 어떤 경우엔 전처리한 데이터가 원본 데이터보다 훨씬 작아서 공간을 절약하고 다운로드 속도도 빨라짐.
>- 전처리 로직마다 전처리된 데이터셋을 생성해야 한다면 여러 가지 전처리 로직을 실험하기 어려움.  
>- 데이터 증강을 하려면 데이터셋의 변종을 미리 만들어야 하므로 디스크 공간이 많이 필요하고 생성 시간도 오래 걸림.
>- 학습된 모델이 전처리된 데이터를 기대하기 때문에 모델을 호출하기 전에 어플리케이션에 전처리 코드를 미리 추가해야 함.


> **tf.data 파이프라인 안에서 전처리할 경우**
> - 전처리 로직을 변경하고 데이터 증식을 적용하기 훨씬 쉬움
> - 매우 효율적인 전처리 파이프라인을 만들 수 있음. (멀티스레딩, 프리페칭)
> - 학습 속도가 느려짐.
> - 데이터 파일을 만들 때 딱 한 번만 데이터를 전처리하는 것이 아니라 매 에포크마다 학습 샘플을 전처리함.
> - 학습된 모델이 여전히 전처리된 데이터를 기대함.


> **모델의 전처리 층에서 전처리할 경우**
>- 학습과 추론을 위해 딱 한 번만 전처리 코드를 작성하면 됨
>- 전처리 로직이 모델의 일부분이기 때문에 모델에 잘못된 전처리 로직을 사용할 위험이 없음.
>- 전처리때문에 학습 속도가 느려지고 학습 샘플이 에포크마다 전처리됨
>- 또한 기본적으로 현재 배치의 전처리 연산이 GPU에서 실행되므로 CPU에서 병렬로 처리하거나 프리페칭을 사용할 수 없다고 함


> **TF 변환을 사용하여 전처리할 경우**
>- 전처리 데이터를 실제로 만들기 때문에 각 샘플이 딱 한 번만 전처리됨 (학습 속도 향상)
>- 전처리 층이 자동으로 생성되므로 전처리 코드를 한 번만 작성하면 됨.
>- 이 도구를 사용하는 방법을 따로 배워야 함.

---
## 8. 범주형 특성을 인코딩할 수 있는 대표적인 방법을 나열하라. 텍스트 데이터는 어떻게 인코딩할 수 있는가?
>**범주형 특성 인코딩**
>- (나쁨, 보통, 좋음)과 같이 자연스러운 순서가 있는 범주형 특성을 인코딩하는 간단한 방법은 **순서가 있는 인코딩**을 사용하는 것. 예를 들어 (나쁨:0, 보통:1, 좋음:2). 
>- 하지만 대부분의 범주형 특성은 내재된 순서가 없음. 이런 경우에는 **원-핫 인코딩(범주 적으면), 임베딩(범주 많으면)**을 이용할 수 있음.

>**텍스트 인코딩**
>- **BoW** 표현을 사용. 문장을 가능한 단어의 카운트 벡터로 표현하는 것. 자주 등장하는 단어는 중요하지 않기 때문에 **TF-IDF**를 사용해 가중치를 낮춤. 
>- 단어를 카운트하는 대신 종종 n개의 단어가 연속된 **n-그램** 을 카운트함.
>- **사전 학습된 단어 임베딩** 을 사용해서 각 단어를 인코딩할 수 있음. 
>- **단어를 인코딩하는 대신 각 글자를 인코딩하거나 부분 토큰을 인코딩**할 수도 있음.